In [0]:
!pip install -q keras

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [4]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1iai6zKxrGYIza4k1Kbwb8eTyD4wybYE8'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()[0:500]))

Downloaded content "﻿CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, ‘and what is the use of a book,’ thought Alice ‘without pictures or
conversations?’

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a da"


# How sequences are generated?  
Instead of using a sliding window of a fixed size, we add more charcters to our sequence and ask the network to predict the next character.The sequences will be padded so that all of them will be of length 100.If there are sentences longer than 100 characters, then they will be broken down into smaller pieces and the above mentioned approah will be applied on each of the smaller pieces.

In [5]:
sentence="jon snow knows nothing"
for i in range(0,len(sentence)):
  seq_in=sentence[0:i]
  seq_out=sentence[i]
  print('input:',seq_in,".output:",seq_out)

input:  .output: j
input: j .output: o
input: jo .output: n
input: jon .output:  
input: jon  .output: s
input: jon s .output: n
input: jon sn .output: o
input: jon sno .output: w
input: jon snow .output:  
input: jon snow  .output: k
input: jon snow k .output: n
input: jon snow kn .output: o
input: jon snow kno .output: w
input: jon snow know .output: s
input: jon snow knows .output:  
input: jon snow knows  .output: n
input: jon snow knows n .output: o
input: jon snow knows no .output: t
input: jon snow knows not .output: h
input: jon snow knows noth .output: i
input: jon snow knows nothi .output: n
input: jon snow knows nothin .output: g


## LSTM Text Generation model

In [0]:
import numpy as np
import string
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences

class TextGenerator:
  
  def __init__(self):
    self.clean_text=None
    self.chars=None
    self.char_to_int=None
    self.sequence_model=None
    self.X=None
    self.y=None
    self.train_history=None
    
    
  def load_and_clean_text(self,file_content):
    
    #converting text to lower case
    self.clean_text=file_content.lower()
    
    #strip all of the new line characters so that we have one long sequence of characters separated only by white space.
    tokens=self.clean_text.split()
    self.clean_text=' '.join(tokens)
    
    #removing punctuations other than full stop
    punctuations_to_remove=''.join([ch for ch in string.punctuation if ch!='.'])
    self.clean_text=self.clean_text.translate(str.maketrans('', '', punctuations_to_remove))
    self.clean_text=self.clean_text.replace('\ufeff'," ").replace('\n'," ").replace('\r'," ").replace( '‘',"").replace('’',"").replace('“',"").replace('”',"").replace("  "," ")
    
    # create mapping of unique chars to integers
    self.chars = sorted(list(set(self.clean_text)))
    
    self.char_to_int = dict((c, i) for i, c in enumerate(self.chars))
    self.int_to_char = dict((i, c) for i, c in enumerate(self.chars))
    
    self.n_chars = len(self.clean_text)
    self.n_vocab = len(self.chars)
    print("Total Characters: ", self.n_chars)
    print("Total Vocab: ", self.n_vocab)
    
    
  def prepare_data_set(self,seq_len=100):
    #prepare the dataset of input to output pairs encoded as integers
    seq_length=seq_len
    dataX=[]
    dataY=[]
    
    sentences=self.clean_text.split('.')
    
    for sentence in sentences:
      if len(sentence)<=seq_length:
        for i in range(0,len(sentence)):
          seq_in=sentence[0:i]
          seq_out=sentence[i]
          dataX.append([self.char_to_int[char] for char in seq_in])
          dataY.append(self.char_to_int[seq_out])
      else:
        smaller_sentences=[sentence[i:i+seq_length] for i in range(0, len(sentence), seq_length)]
        for smaller_sentence in smaller_sentences:
          if len(smaller_sentence)<=seq_length:
            for i in range(0,len(smaller_sentence)):
              seq_in=smaller_sentence[0:i]
              seq_out=smaller_sentence[i]
              dataX.append([self.char_to_int[char] for char in seq_in])
              dataY.append(self.char_to_int[seq_out])
              
      
    n_patterns = len(dataX)
    print("Total Patterns: ", n_patterns)
    dataX_padded=np.array(pad_sequences(dataX, maxlen=seq_length, padding='pre'))
    
    print('shape of dataX_padded:',dataX_padded.shape)
    # reshape X to be [samples, time steps, features]
    self.X = np.reshape(dataX_padded, (n_patterns, seq_length, 1))
    # normalize
    self.X = self.X / float(self.n_vocab)
    # one hot encode the output variable
    self.y = np_utils.to_categorical(dataY)
    
    
  def create_sequence_model(self,num_lstm_units=256,drop_out=0.1):
    model = Sequential()
    model.add(LSTM(num_lstm_units, input_shape=(self.X.shape[1],self.X.shape[2]),dropout=drop_out,return_sequences=True))
    model.add(LSTM(num_lstm_units))
    model.add(Dense(self.y.shape[1], activation='softmax'))
    
    self.sequence_model=model
    self.sequence_model.summary()
    
  def train_model(self,file_path_to_store_weights,
                  num_epochs=100,
                  batch_size=128,
                 loss_fn='categorical_crossentropy',
                 optimizer_name='adam'):
    
    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=file_path_to_store_weights,
                             monitor='loss',
                             verbose=1,
                             save_weights_only=False,
                             save_best_only=False,
                             mode='min',
                             period=10)
    
    callbacks_list = [checkpoint]
    
    self.sequence_model.compile(loss=loss_fn, optimizer=optimizer_name)
    # fit the model
    self.train_history=self.sequence_model.fit(self.X, self.y, epochs=num_epochs, batch_size=batch_size, callbacks=callbacks_list)
    

  def predict(self,char_seed='cat',num_chars_to_predict=100):
    char_prediction=char_seed
    pattern = [self.char_to_int[ch] for ch in char_seed]
    print("Seed:",char_seed)
  
    # generate characters
    for i in range(num_chars_to_predict):
      x = np.array(pad_sequences([pattern], maxlen=100, padding='pre'))
      x=np.reshape(x, (1, 100, 1))
      x = x / float(self.n_vocab)
      prediction = self.sequence_model.predict(x, verbose=0)
      index = np.argmax(prediction)
      result = self.int_to_char[index]
      seq_in = [self.int_to_char[value] for value in pattern]
      char_prediction+=result
      pattern.append(index)
      pattern = pattern[1:len(pattern)]
  
    return char_prediction
  
  
  def predict_with_temperature(self,char_seed='cat',num_chars_to_predict=100,temperature=1):
    char_prediction=char_seed
    pattern = [self.char_to_int[ch] for ch in char_seed]
#     print("Seed:",char_seed)
  
    # generate characters
    for i in range(num_chars_to_predict):
      x = np.array(pad_sequences([pattern], maxlen=100, padding='pre'))
      x=np.reshape(x, (1, 100, 1))
      x = x / float(self.n_vocab)
      prediction = self.sequence_model.predict(x, verbose=0)
      
      preds=np.asarray(prediction).astype('float64')
      preds=np.log(preds)/temperature
      index = np.argmax(preds)
      
      result = self.int_to_char[index]
      seq_in = [self.int_to_char[value] for value in pattern]
      char_prediction+=result
      pattern.append(index)
      pattern = pattern[1:len(pattern)]
  
    return char_prediction 
  

In [0]:
txt_generator=TextGenerator()
txt_generator.load_and_clean_text(downloaded.GetContentString())

Total Characters:  135043
Total Vocab:  28


In [0]:
txt_generator.prepare_data_set(seq_len=100)

Total Patterns:  134054
shape of dataX_padded: (134054, 100)


In [0]:
txt_generator.create_sequence_model(num_lstm_units=256,drop_out=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 28)                7196      
Total params: 796,700
Trainable params: 796,700
Non-trainable params: 0
_________________________________________________________________


In [0]:
weights_save_path='/content/gdrive/My Drive/EIP/Phase2/Assignment2/weights/approach1_v5/July_26/epochs_001_100:{epoch:03d}.hdf5'
txt_generator.train_model(file_path_to_store_weights=weights_save_path,num_epochs=100,batch_size=512)

Epoch 1/100
134054/134054 [==============================] - 295s 2ms/step - loss: 2.8186
Epoch 2/100
134054/134054 [==============================] - 294s 2ms/step - loss: 2.6504
Epoch 3/100
134054/134054 [==============================] - 295s 2ms/step - loss: 2.4264
Epoch 4/100
134054/134054 [==============================] - 294s 2ms/step - loss: 2.2695
Epoch 5/100
134054/134054 [==============================] - 294s 2ms/step - loss: 2.1514
Epoch 6/100
134054/134054 [==============================] - 295s 2ms/step - loss: 2.0611
Epoch 7/100
134054/134054 [==============================] - 294s 2ms/step - loss: 1.9816
Epoch 8/100
134054/134054 [==============================] - 294s 2ms/step - loss: 1.9183
Epoch 9/100
134054/134054 [==============================] - 294s 2ms/step - loss: 1.8636
Epoch 10/100
134054/134054 [==============================] - 294s 2ms/step - loss: 1.8139

Epoch 00010: saving model to /content/gdrive/My Drive/EIP/Phase2/Assignment2/weights/approach1_v5/

##Epochs 81 to 100  
We will retrain the model from epoch 81.  

In [15]:
txt_generator=TextGenerator()
txt_generator.load_and_clean_text(downloaded.GetContentString())

Total Characters:  135043
Total Vocab:  28


In [16]:
txt_generator.prepare_data_set(seq_len=100)

Total Patterns:  134054
shape of dataX_padded: (134054, 100)


In [17]:
txt_generator.create_sequence_model(num_lstm_units=256,drop_out=0.1)
txt_generator.sequence_model.load_weights("/content/gdrive/My Drive/EIP/Phase2/Assignment2/weights/approach1_v5/July_26/epochs_001_100:080.hdf5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_3 (Dense)              (None, 28)                7196      
Total params: 796,700
Trainable params: 796,700
Non-trainable params: 0
_________________________________________________________________


In [18]:
weights_save_path='/content/gdrive/My Drive/EIP/Phase2/Assignment2/weights/approach1_v5/July_26/epochs_081_100:{epoch:03d}.hdf5'
txt_generator.train_model(file_path_to_store_weights=weights_save_path,num_epochs=20,batch_size=512)

Epoch 1/20
134054/134054 [==============================] - 159s 1ms/step - loss: 0.5184
Epoch 2/20
134054/134054 [==============================] - 158s 1ms/step - loss: 0.5034
Epoch 3/20
134054/134054 [==============================] - 156s 1ms/step - loss: 0.4936
Epoch 4/20
134054/134054 [==============================] - 155s 1ms/step - loss: 0.4958
Epoch 5/20
134054/134054 [==============================] - 156s 1ms/step - loss: 0.4831
Epoch 6/20
134054/134054 [==============================] - 157s 1ms/step - loss: 0.4777
Epoch 7/20
134054/134054 [==============================] - 155s 1ms/step - loss: 0.4688
Epoch 8/20
134054/134054 [==============================] - 159s 1ms/step - loss: 0.4689
Epoch 9/20
134054/134054 [==============================] - 158s 1ms/step - loss: 0.4620
Epoch 10/20
134054/134054 [==============================] - 159s 1ms/step - loss: 0.4529

Epoch 00010: saving model to /content/gdrive/My Drive/EIP/Phase2/Assignment2/weights/approach1_v5/July_26/ep

## Predicting 500 characters

In [27]:
txt_generator=TextGenerator()
txt_generator.load_and_clean_text(downloaded.GetContentString())
txt_generator.prepare_data_set(seq_len=100)
txt_generator.create_sequence_model(num_lstm_units=256,drop_out=0.1)
txt_generator.sequence_model.load_weights("/content/gdrive/My Drive/EIP/Phase2/Assignment2/weights/approach1_v5/July_26/epochs_081_100:020.hdf5")
txt_generator.sequence_model.compile(loss='categorical_crossentropy', optimizer='adam')

Total Characters:  135043
Total Vocab:  28
Total Patterns:  134054
shape of dataX_padded: (134054, 100)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
lstm_8 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_4 (Dense)              (None, 28)                7196      
Total params: 796,700
Trainable params: 796,700
Non-trainable params: 0
_________________________________________________________________


In [28]:
prediction=txt_generator.predict(char_seed='alice wants to',num_chars_to_predict=500)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

Seed: alice wants to
alice wants to meared afrarteraledped ly oow gorustaod bnd tmmed whats note thal thely uif whought a
llce iilgs arowher degtcde anice asked ln a ponee of ier ase teepv it anl clovs ttmce harde anl roe 
sffgly iad ntrked anl round again to her iil sichly iad asmeoonoe bllce hasdly gear whe corksr anice
 had froe the in had frigd thale io the whoug tlees wher saws so alice hade been an ofst tte clngsaw
 w vningnngxt waid the lnut tuested hell and a somaee of pne said alice anrnget hestet wn her donse the hreat qut 


In [29]:
prediction=txt_generator.predict_with_temperature(char_seed='alice wants to',num_chars_to_predict=500,temperature=0.2)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

alice wants to meared afrarteraledped ly oow gorustaod bnd tmmed whats note thal thely uif whought a
llce iilgs arowher degtcde anice asked ln a ponee of ier ase teepv it anl clovs ttmce harde anl roe 
sffgly iad ntrked anl round again to her iil sichly iad asmeoonoe bllce hasdly gear whe corksr anice
 had froe the in had frigd thale io the whoug tlees wher saws so alice hade been an ofst tte clngsaw
 w vningnngxt waid the lnut tuested hell and a somaee of pne said alice anrnget hestet wn her donse the hreat qut 


In [30]:
prediction=txt_generator.predict_with_temperature(char_seed='alice wants to',num_chars_to_predict=500,temperature=0.5)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

alice wants to meared afrarteraledped ly oow gorustaod bnd tmmed whats note thal thely uif whought a
llce iilgs arowher degtcde anice asked ln a ponee of ier ase teepv it anl clovs ttmce harde anl roe 
sffgly iad ntrked anl round again to her iil sichly iad asmeoonoe bllce hasdly gear whe corksr anice
 had froe the in had frigd thale io the whoug tlees wher saws so alice hade been an ofst tte clngsaw
 w vningnngxt waid the lnut tuested hell and a somaee of pne said alice anrnget hestet wn her donse the hreat qut 


In [31]:
prediction=txt_generator.predict_with_temperature(char_seed='alice wants to',num_chars_to_predict=500,temperature=1)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

alice wants to meared afrarteraledped ly oow gorustaod bnd tmmed whats note thal thely uif whought a
llce iilgs arowher degtcde anice asked ln a ponee of ier ase teepv it anl clovs ttmce harde anl roe 
sffgly iad ntrked anl round again to her iil sichly iad asmeoonoe bllce hasdly gear whe corksr anice
 had froe the in had frigd thale io the whoug tlees wher saws so alice hade been an ofst tte clngsaw
 w vningnngxt waid the lnut tuested hell and a somaee of pne said alice anrnget hestet wn her donse the hreat qut 


In [32]:
prediction=txt_generator.predict_with_temperature(char_seed='alice wants to',num_chars_to_predict=500,temperature=1.2)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

alice wants to meared afrarteraledped ly oow gorustaod bnd tmmed whats note thal thely uif whought a
llce iilgs arowher degtcde anice asked ln a ponee of ier ase teepv it anl clovs ttmce harde anl roe 
sffgly iad ntrked anl round again to her iil sichly iad asmeoonoe bllce hasdly gear whe corksr anice
 had froe the in had frigd thale io the whoug tlees wher saws so alice hade been an ofst tte clngsaw
 w vningnngxt waid the lnut tuested hell and a somaee of pne said alice anrnget hestet wn her donse the hreat qut 


In [33]:
prediction=txt_generator.predict(char_seed='jon snow knows nothing',num_chars_to_predict=500)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

Seed: jon snow knows nothing
jon snow knows nothing fay he conled inm foinveny trum vhe vhoug ture but semember tuestellng what a
 momgnt as she whole party suall ier agr anl hall and must tueak das cett inttant inl stuppper whe m
ing said cytinngng soand when suder waiting tisters and muoked suite po wie toatcrpl it vhingseatt i
t whink as i meare ier wail an of vathesdst anxiouse and the sathen deutainly said to the hell asked
 alice had tomnkng again and sulle the whoskeeisgollfve imp crmfnd deingst tlg in the sawher suadenlng taid to tee it agra


In [34]:
prediction=txt_generator.predict_with_temperature(char_seed='jon snow knows nothing',num_chars_to_predict=500,temperature=0.2)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

jon snow knows nothing fay he conled inm foinveny trum vhe vhoug ture but semember tuestellng what a
 momgnt as she whole party suall ier agr anl hall and must tueak das cett inttant inl stuppper whe m
ing said cytinngng soand when suder waiting tisters and muoked suite po wie toatcrpl it vhingseatt i
t whink as i meare ier wail an of vathesdst anxiouse and the sathen deutainly said to the hell asked
 alice had tomnkng again and sulle the whoskeeisgollfve imp crmfnd deingst tlg in the sawher suadenlng taid to tee it agra


In [35]:
prediction=txt_generator.predict_with_temperature(char_seed='jon snow knows nothing',num_chars_to_predict=500,temperature=0.5)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

jon snow knows nothing fay he conled inm foinveny trum vhe vhoug ture but semember tuestellng what a
 momgnt as she whole party suall ier agr anl hall and must tueak das cett inttant inl stuppper whe m
ing said cytinngng soand when suder waiting tisters and muoked suite po wie toatcrpl it vhingseatt i
t whink as i meare ier wail an of vathesdst anxiouse and the sathen deutainly said to the hell asked
 alice had tomnkng again and sulle the whoskeeisgollfve imp crmfnd deingst tlg in the sawher suadenlng taid to tee it agra


In [36]:
prediction=txt_generator.predict_with_temperature(char_seed='jon snow knows nothing',num_chars_to_predict=500,temperature=1)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

jon snow knows nothing fay he conled inm foinveny trum vhe vhoug ture but semember tuestellng what a
 momgnt as she whole party suall ier agr anl hall and must tueak das cett inttant inl stuppper whe m
ing said cytinngng soand when suder waiting tisters and muoked suite po wie toatcrpl it vhingseatt i
t whink as i meare ier wail an of vathesdst anxiouse and the sathen deutainly said to the hell asked
 alice had tomnkng again and sulle the whoskeeisgollfve imp crmfnd deingst tlg in the sawher suadenlng taid to tee it agra


In [37]:
prediction=txt_generator.predict_with_temperature(char_seed='jon snow knows nothing',num_chars_to_predict=500,temperature=1.2)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

jon snow knows nothing fay he conled inm foinveny trum vhe vhoug ture but semember tuestellng what a
 momgnt as she whole party suall ier agr anl hall and must tueak das cett inttant inl stuppper whe m
ing said cytinngng soand when suder waiting tisters and muoked suite po wie toatcrpl it vhingseatt i
t whink as i meare ier wail an of vathesdst anxiouse and the sathen deutainly said to the hell asked
 alice had tomnkng again and sulle the whoskeeisgollfve imp crmfnd deingst tlg in the sawher suadenlng taid to tee it agra


## Some more examples

In [38]:
prediction=txt_generator.predict(char_seed='a lannister always',num_chars_to_predict=500)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

Seed: a lannister always
a lannister always waid the rabbitsrrt tucnd pf the addusachabt shasly and fo hntotsw tdiaid iv apd 
senarked tome oiedly put hou temame ovt at a grcit all mysty townded heard yould klehes delply temar
ke imayong the hot dried tationing sepldpy toall bgain and said clttoaone oi the lowide holng hear b
 bycy weacypionsety hasd an i tueatly how bny hav bats taid to tisw its chml tff hoodeaathonses anic
e coashed alas the waie the hattinler ttupidts alice wholeles suestery and mnoked suasrellng anl the wented of a grode


In [39]:
prediction=txt_generator.predict(char_seed='the meaning of life is',num_chars_to_predict=500)
print(prediction[0:100])
print(prediction[100:200])
print(prediction[200:300])
print(prediction[300:400])
print(prediction[400:])

Seed: the meaning of life is
the meaning of life is a vort of crokusts tuaane asp at all eire the nosking oo roe teguhd hird it a
ll todes tharply about hot thg way onv suu it made ir ooce as she looked suuping swier tealing hir v
aid alice and i whinking the wholeysee the had frund taid the ling a boovery of whem i canled uumori
te toale and thonted at round as i uaidnt wtimt to tee if whoug to agteed the lfaving whem and the w
hole put bumsset what whan im the wail tus it to a soees the wholey toasmellgoy taid to the halfeued heresale iye kunger a
